In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

spark = SparkSession.builder \
    .appName("experimentation-data-icebertg") \
    .getOrCreate()




26/01/09 16:27:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
name_basics_df = spark.table("demo.bronze.name_basics")
name_basics_df.show(5)


+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0072308,tt00504...|
|nm0000002|  Lauren Bacall|     1924|     2014|actress,miscellan...|tt0037382,tt00752...|
|nm0000003|Brigitte Bardot|     1934|     2025|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00694...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows



In [3]:
title_akas_df = spark.table("demo.bronze.title_akas")
title_akas_df.show()


+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Carmencita|    \N|      \N|   original|                  \N|              1|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|       literal title|              0|
|tt0000001|       3|          Carmencita|    US|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       4|Carmencita - span...|    HU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       5|          Καρμενσίτα|    GR|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       6|          Карменсита|    RU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       7|          Карменс

In [4]:
title_basics_df = spark.table("demo.bronze.title_basics")
title_basics_df.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|              

In [5]:
title_crew_df = spark.table("demo.bronze.title_crew")
title_crew_df.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `demo`.`golden`.`title_crew` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [demo, golden, title_crew], [], false


In [6]:
title_episode_df = spark.table("demo.bronze.title_episode")
title_episode_df.show()

+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0031458|  tt32857063|          \N|           \N|
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
|tt0042889|   tt0989125|          \N|           \N|
|tt0043426|   tt0040051|           3|           42|
|tt0043631|   tt0989125|           2|           16|
|tt0043693|   tt0989125|           2|            8|
|tt0043710|   tt0989125|           3|            3|
|tt0044093|   tt0959862|           1|            6|
|tt0044668|   tt0044243|           2|           16|
|tt0044901|   tt0989125|           3|           46|
|tt0045519|   tt0989125|           4|           11|
|tt0045960|   tt0044284|           2|            3|
|tt0046135|   tt0989125|           4|            5|
|tt0046150|   tt0341798|          \N|           \N|
|tt0046855|   tt0046643|           1|            4|
|tt0046864| 

In [7]:
title_principals_df = spark.table("demo.bronze.title_principals")
title_principals_df.show()

+---------+--------+---------+---------------+--------------------+--------------+
|   tconst|ordering|   nconst|       category|                 job|    characters|
+---------+--------+---------+---------------+--------------------+--------------+
|tt0000001|       1|nm1588970|           self|                  \N|      ["Self"]|
|tt0000001|       2|nm0005690|       director|                  \N|            \N|
|tt0000001|       3|nm0005690|       producer|            producer|            \N|
|tt0000001|       4|nm0374658|cinematographer|director of photo...|            \N|
|tt0000002|       1|nm0721526|       director|                  \N|            \N|
|tt0000002|       2|nm1335271|       composer|                  \N|            \N|
|tt0000003|       1|nm0721526|       director|                  \N|            \N|
|tt0000003|       2|nm0721526|         writer|                  \N|            \N|
|tt0000003|       3|nm1770680|       producer|            producer|            \N|
|tt0

In [8]:
title_ratings_df = spark.table("demo.bronze.title_ratings")
title_ratings_df.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2187|
|tt0000002|          5.5|     307|
|tt0000003|          6.5|    2280|
|tt0000004|          5.1|     196|
|tt0000005|          6.2|    3013|
|tt0000006|          5.1|     220|
|tt0000007|          5.3|     933|
|tt0000008|          5.4|    2361|
|tt0000009|          5.2|     232|
|tt0000010|          6.8|    8180|
|tt0000011|          5.2|     438|
|tt0000012|          7.4|   13826|
|tt0000013|          5.7|    2112|
|tt0000014|          7.1|    6318|
|tt0000015|          6.1|    1304|
|tt0000016|          5.9|    1701|
|tt0000017|          4.6|     395|
|tt0000018|          5.2|     687|
|tt0000019|          4.7|      37|
|tt0000020|          4.7|     424|
+---------+-------------+--------+
only showing top 20 rows

